In [ ]:
import numpyro
from numpyro import sample, handlers, plate
import numpyro.distributions as dist
from jax import random
from numpyro.infer import MCMC, NUTS
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
import pandas as pd
import json

from scipy.special import ndtr
ndtr = lambda x: 1 / ( 1 + jnp.exp(x))

In [ ]:
def make_img_idx(df):
    img_idx = [None] * 7
    c = 0
    for i in range(7):
        for j in range(i+1, 7):
            img_idx[i] = df.index[c][0]
            img_idx[j] = df.index[c][1]
            c += 1
    return img_idx

In [ ]:
data_dir = '../data/processed/'
with open(data_dir + 'stimulus_wr.json') as fh:
    stimulus_wr = json.load(fh)
# pd.read_pickle('pooled_data.pkl')

# df = pd.read_hdf('pooled_data.hdf')
# df.sum(axis=1)
# # stimulus_wr
# # df
# # df.index
# img_idx = make_img_idx(df)
# x = [stimulus_wr[key] for key in img_idx]
# x
# # df

In [ ]:
def latent_variable_model(obs, N):
    with plate('i', 7):
        mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(len(mu)):
        for j in range(i+1, len(mu)):
            sample(f'diff_{i}{j}', dist.Binomial(N[i], ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1

# nuts_kernel = NUTS(latent_variable_model)
# mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
# rng_key = random.PRNGKey(0)
# mcmc.run(rng_key, df.win2, df.sum(axis=1))

# s = mcmc.get_samples()
# est_mu = s['mu'].mean(axis=0)

In [ ]:
prefix = 'experiment'
s = {}
for g in ['low', 'medium', 'high']:
    name = data_dir + f'pooled_{prefix}_g-{g}.pkl'
    data = pd.read_pickle(name)
    
    nuts_kernel = NUTS(latent_variable_model)
    mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
    rng_key = random.PRNGKey(0)
    mcmc.run(rng_key, data.win2, data.sum(axis=1))
    s[g] = mcmc.get_samples()

In [ ]:
g2c = {
    'low': 'red',
    'medium': 'blue',
    'high': 'green'
}
def parabola(x, w):
    return w[0] + w[1] * x + w[2] * x**2

def wr_to_thurstone(t, y=None):
    with plate('params', 3):
        w = sample('w', dist.Normal(0, 10000))
    mu = w[0] + w[1] * t + w[2] * t ** 2
    sigma = sample('sigma', dist.Uniform(0, 10))
    return sample('y', dist.Normal(mu, sigma), obs=y)

In [ ]:
r = 0
with handlers.seed(rng_seed=r):
    print(wr_to_thurstone(0, 1))
    r+=1

In [ ]:
ws = {}
wmle = {}
for g in ['low', 'medium', 'high']:
    plt.vlines(stimulus_wr[f'avgimg_wr_{g}.png'], -2, 1.6, linestyles='--', alpha=0.7, color=g2c[g])
    name = data_dir + f'pooled_{prefix}_g-{g}.pkl'
    data = pd.read_pickle(name)
    img_idx = make_img_idx(data)
    x = np.array([stimulus_wr[key] for key in img_idx])
    est_mu = s[g]['mu'].mean(axis=0)
    idxs = x.argsort()
    
    points = [(x, y) for x, y in zip(x, est_mu)]
    print(points)
    # w = least_squares(partial(pcost, points), [0, 0, 0]).x
    X = np.array([p[0] for p in points])
    Y = jnp.array([p[1] for p in points])
    A = np.vstack([np.ones(len(X)), X, X**2]).T
    wmle[g] = np.linalg.lstsq(A, [p[1] for p in points])[0]
    #x_test = np.linspace(0.43, 0.49)
    x_test = np.linspace(x.min()-0.005, x.max()+0.005)
    y = parabola(x_test, wmle[g])
    plt.plot(x_test, y, color=g2c[g], alpha=1)
    
    nuts_kernel = NUTS(wr_to_thurstone)
    mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
    rng_key = random.PRNGKey(0)
    mcmc.run(rng_key, X, Y)
    ws[g] = mcmc.get_samples()
    
    for i in range(100):
        plt.plot(x_test, parabola(x_test, ws[g]['w'][-i]), color=g2c[g], alpha=0.05)
    
    mu = s[g]['mu']
    for i in range(500):
        mu_sampled = mu[-i]
        plt.plot(x, mu_sampled, '.', alpha=0.02, color=g2c[g])
    plt.plot(x[idxs], est_mu[idxs], '+', color=g2c[g], label=g)
plt.legend()

In [ ]:
import seaborn as sns
sns.set_theme()
fig, ax = plt.subplots()
fig.set_size_inches((11, 6))
for g in ['low', 'medium', 'high']:
    plt.vlines(stimulus_wr[f'avgimg_wr_{g}.png'], -2, 1.6, linestyles='--', alpha=0.7, color=g2c[g])
    name = data_dir + f'pooled_{prefix}_g-{g}.pkl'
    data = pd.read_pickle(name)
    img_idx = make_img_idx(data)
    x = np.array([stimulus_wr[key] for key in img_idx])
    est_mu = s[g]['mu'].mean(axis=0)
    idxs = x.argsort()
    
    points = [(x, y) for x, y in zip(x, est_mu)]
    X = np.array([p[0] for p in points])
    Y = jnp.array([p[1] for p in points])
    A = np.vstack([np.ones(len(X)), X, X**2]).T
    #x_test = np.linspace(x.min()-0.005, x.max()+0.005)
    x_test = np.linspace(0.4, 0.6, 100)
    y = parabola(x_test, wmle[g])
    plt.plot(x_test, y, color=g2c[g], alpha=1)
    mx = -wmle[g][1] / 2 / wmle[g][2]
    plt.vlines(mx, parabola(mx, wmle[g]), 1.5, linestyles='-.', color=g2c[g], alpha=0.75)
    
    ridx = np.random.choice(range(len(ws[g]['w'])), 70)
    for i in ridx:
        plt.plot(x_test, parabola(x_test, ws[g]['w'][i]), color=g2c[g], alpha=0.05)
        mx = -ws[g]['w'][i, 1] / 2 / ws[g]['w'][i, 2]
        #plt.vlines(mx, parabola(mx, ws[g]['w'][i]), 1.5, color=g2c[g], alpha=0.05)

    mx = -ws[g]['w'][:, 1] / 2 / ws[g]['w'][:, 2]
    plt.plot(mx[ridx], 1.5 * np.ones_like(ridx), '*', ms=3, color=g2c[g], alpha=0.05)
    
    mu = s[g]['mu']
    for i in range(500):
        mu_sampled = mu[-i]
        plt.plot(x, mu_sampled, '.', alpha=0.02, color=g2c[g])
    plt.plot(x[idxs], est_mu[idxs], '+', color=g2c[g], label=g)
plt.ylabel('Thurstone Score [a.u.]')
plt.xlabel('Width Ratio')
plt.ylim(-2, 1.6)
plt.xlim(0.43, 0.505)
plt.legend()
plt.tight_layout()
plt.savefig('../data/result.pdf')

In [ ]:
stimulus_wr

In [ ]:
def parabel(wr, a, b, c):
    return a + b * wr + c * wr ** 2

def latent_variable_model_with_parabola(wr, obs, N_comparisons):
    a = sample('a', dist.Normal(0, 10000))
    #m = sample('m', dist.Normal(0.46, 0.1))
    b = sample('b', dist.Normal(0, 10000))
    c = sample('c', dist.Normal(0, 10000))
    #c = numpyro.deterministic('c', -b/2/m)
    sigma = sample('sigma', dist.Uniform(0., 10.))
    #sigma = 0.01
    N = len(wr)
    mu = [sample(f'mu_{i}', dist.Normal(parabel(wr[i], a, b, c), sigma)) for i in range(N)]
    # mu = [sigma * sample(f'mu_{i}', dist.Normal(0, 1)) + parabel(wr[i], a, b, c)  for i in range(N)]
    #with plate('i', len(test_ranking)):
        #mu = sample('mu', dist.Normal(0, 1))
    c = 0
    for i in range(N):
        for j in range(i+1, N):
            sample(f'diff_{i}{j}', dist.Binomial(N_comparisons[c], ndtr(mu[i] - mu[j])), obs=obs[c])
            c += 1
            
g = 'medium'
for g in ['low', 'medium', 'high']:
    name = data_dir + f'pooled_{prefix}_g-{g}.pkl'
    data = pd.read_pickle(name)

    img_idx = make_img_idx(data)
    x = np.array([stimulus_wr[key] for key in img_idx])
    # x = (x - x.mean()) / x.std()
    idxs = x.argsort()

    # points = [(x, y) for x, y in zip(x, est_mu)]

    # X = np.array([p[0] for p in points])
    # Y = jnp.array([p[1] for p in points])


    nuts_kernel = NUTS(latent_variable_model_with_parabola)
    mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
    rng_key = random.PRNGKey(0)
    mcmc.run(rng_key, x, data.win2, data.sum(axis=1))
    samples = mcmc.get_samples()

    x_test = np.linspace(x.min()-0.005, x.max()+0.005)
    for i in range(100, 150):
        y_test = parabel(x_test, 0, samples['b'][i], samples['c'][i])
        #y_test = parabel(x_test, samples['a'][i], samples['b'][i], samples['c'][i], alpha=0.3)
        plt.plot(x_test, y_test, alpha=0.3)
    m = -samples['b'].mean() / 2 / samples['c'].mean()
    print(m)
    plt.plot(m, 0, 'x')

In [ ]:
x

In [ ]:
# np.array(x).argsort()
# stimulus_wr
mx = -ws['low']['w'][:, 1] / 2 / ws['low']['w'][:, 2]
np.random.choice(mx, 5)
data.to_excel('data_table.xlsx')